# Desafio Laboratório Prático Final
###### Tópicos em Banco de Dados
======


In [4]:
# instalando dependências necessárias
!pip install pymongo

## Configurando bibliotecas e estruturas padrão para o projeto

In [5]:
# bibliotecas de aprendizado de máquina e mineração de dados
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import re
import nltk
from nltk.corpus import stopwords
import tqdm
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
import string

## Operação para colocar o csv no MongoDB, além de recupera-los e colocar em uma variável para o próximo passo

In [1]:
import csv
from pymongo import MongoClient
from bson.objectid import ObjectId # Importar ObjectId para lidar com o campo _id

# --- Configurações do MongoDB ---
MONGO_HOST = '127.0.0.1' # Ou o endereço IP do seu servidor MongoDB
MONGO_PORT = 27017       # Porta padrão do MongoDB
DB_NAME = 'minha_base_de_dados' # Nome do banco de dados
COLLECTION_NAME = 'meus_dados_csv' # Nome da coleção onde os dados serão inseridos

# --- Credenciais do MongoDB ---
# ATENÇÃO: Em um ambiente de produção, NÃO armazene senhas diretamente no código.
# Use variáveis de ambiente ou um sistema de gerenciamento de segredos.
MONGO_USER = 'admin'
MONGO_PASS = '123456'

# --- Caminho do arquivo CSV ---
CSV_FILE_PATH = 'df_file.csv' # Certifique-se de que este arquivo está no mesmo diretório do seu script Python ou forneça o caminho completo

def ler_csv_e_inserir_no_mongodb(csv_file, db_name, collection_name, host, port, user, password):
    """
    Lê um arquivo CSV e insere seus dados em uma coleção MongoDB,
    com autenticação.

    Args:
        csv_file (str): O caminho para o arquivo CSV.
        db_name (str): O nome do banco de dados no MongoDB.
        collection_name (str): O nome da coleção onde os dados serão inseridos.
        host (str): O host do servidor MongoDB.
        port (int): A porta do servidor MongoDB.
        user (str): Nome de usuário para autenticação no MongoDB.
        password (str): Senha para autenticação no MongoDB.
    """
    client = None
    try:
        # Conecta ao MongoDB usando autenticação na string de conexão URI
        mongo_uri = f"mongodb://{user}:{password}@{host}:{port}/"
        client = MongoClient(mongo_uri)

        # Opcional: Para verificar se a autenticação foi bem-sucedida,
        # você pode tentar acessar um recurso, como o `admin` db.
        # client.admin.command('ismaster') # Isso pode levantar uma exceção se a autenticação falhar

        db = client[db_name]
        collection = db[collection_name]
        print(f"Conectado ao MongoDB em {host}:{port} com autenticação.")

        with open(csv_file, mode='r', encoding='utf-8') as file:
            csv_reader = csv.DictReader(file)
            data_to_insert = []
            for row in csv_reader:
                data_to_insert.append(row)

            if data_to_insert:
                result = collection.insert_many(data_to_insert)
                print(f"Dados inseridos com sucesso! IDs dos documentos: {result.inserted_ids}")
            else:
                print("Nenhum dado encontrado no arquivo CSV para inserir.")

    except FileNotFoundError:
        print(f"Erro: O arquivo '{csv_file}' não foi encontrado.")
    except Exception as e:
        print(f"Ocorreu um erro ao inserir dados ou conectar (verifique credenciais/servidor): {e}")
    finally:
        if client:
            client.close()
            print("Conexão com o MongoDB fechada (após inserção).")

def pegar_dados_do_mongodb_para_dicionario(db_name, collection_name, host, port, user, password):
    """
    Pega todos os documentos de uma coleção MongoDB e os retorna como uma lista de dicionários,
    com autenticação.

    Args:
        db_name (str): O nome do banco de dados no MongoDB.
        collection_name (str): O nome da coleção de onde os dados serão recuperados.
        host (str): O host do servidor MongoDB.
        port (int): A porta do servidor MongoDB.
        user (str): Nome de usuário para autenticação no MongoDB.
        password (str): Senha para autenticação no MongoDB.

    Returns:
        list: Uma lista de dicionários, onde cada dicionário representa um documento da coleção.
              Retorna uma lista vazia se nenhum dado for encontrado ou se ocorrer um erro.
    """
    client = None
    data_from_mongo = []
    try:
        # Conecta ao MongoDB usando autenticação na string de conexão URI
        mongo_uri = f"mongodb://{user}:{password}@{host}:{port}/"
        client = MongoClient(mongo_uri)

        db = client[db_name]
        collection = db[collection_name]
        print(f"Conectado ao MongoDB em {host}:{port} para recuperação de dados com autenticação.")

        for document in collection.find({}):
            if '_id' in document and isinstance(document['_id'], ObjectId):
                document['_id'] = str(document['_id'])
            data_from_mongo.append(document)

        print(f"Recuperou {len(data_from_mongo)} documentos da coleção '{collection_name}'.")

    except Exception as e:
        print(f"Ocorreu um erro ao recuperar dados do MongoDB ou conectar (verifique credenciais/servidor): {e}")
    finally:
        if client:
            client.close()
            print("Conexão com o MongoDB fechada (após recuperação).")
    return data_from_mongo

# --- Chamada das funções principais ---
if __name__ == "__main__":
    # Primeiro, opcionalmente, insira os dados do CSV (se ainda não o fez)
    # Certifique-se de que o usuário 'admin' com a senha '123456'
    # tem as permissões adequadas no MongoDB para o 'minha_base_de_dados'.
    ler_csv_e_inserir_no_mongodb(
        CSV_FILE_PATH, DB_NAME, COLLECTION_NAME, MONGO_HOST, MONGO_PORT,
        MONGO_USER, MONGO_PASS
    )

    # Agora, pegue os dados da coleção
    dados_recuperados = pegar_dados_do_mongodb_para_dicionario(
        DB_NAME, COLLECTION_NAME, MONGO_HOST, MONGO_PORT,
        MONGO_USER, MONGO_PASS
    )

    if dados_recuperados:
        print("\nDados recuperados do MongoDB:")
        for item in dados_recuperados:
            print(item)
    else:
        print("\nNenhum dado foi recuperado do MongoDB.")



Conectado ao MongoDB em 127.0.0.1:27017 com autenticação.
Ocorreu um erro ao inserir dados ou conectar (verifique credenciais/servidor): 127.0.0.1:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 684482ab751e1257c0eeabf7, topology_type: Unknown, servers: [<ServerDescription ('127.0.0.1', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('127.0.0.1:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>
Conexão com o MongoDB fechada (após inserção).
Conectado ao MongoDB em 127.0.0.1:27017 para recuperação de dados com autenticação.
Ocorreu um erro ao recuperar dados do MongoDB ou conectar (verifique credenciais/servidor): 127.0.0.1:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <

# Adicionando aprendizado de máquina sobre a amostra coletada

In [14]:
from warnings import filterwarnings
filterwarnings('ignore')
pd.set_option('display.max_colwidth', 100)

# df = pd.read_csv('./df_file.csv') # encoding='latin-1'
df = dados_recuperados
df.head()
df.info()
df.shape
df.Label.unique()
df.duplicated().sum()

df.drop_duplicates(keep='last', inplace=True)
df.duplicated().sum()

df.isnull().values.any()

df.head()

df.replace({0:"Politics", 1:"Sport", 2:"Technology", 3:"Entertainment", 4:"Business"}, inplace=True)

df.Label.value_counts()

AttributeError: 'list' object has no attribute 'head'

In [ ]:
import plotly.express as px
import pandas as pd

# Contagem de valores da coluna 'Label'
label_counts = df['Label'].value_counts().reset_index()
label_counts.columns = ['Label', 'Count']

# Gráfico de pizza com buraco (do tipo donut)
fig = px.pie(label_counts,
             values='Count',
             names='Label',
             hole=0.5,
             title='Label distribution of the Text')

fig.show()

In [ ]:
df.head()

df['Text'][df['Label'] == 'Politics'][0]

In [ ]:
def cleanTweets(text):
    txt = text.lower() #convert all letters to lower case
    txt = re.sub("@[A-Za-z0-9_]+","", txt) #remove mentions
    txt = re.sub(r"http\S+", "", txt) #remove url's https tags
    txt = re.sub(r"www.\S+", "", txt) #remove url's www tags
    txt = re.sub("[^a-z0-9]"," ", txt) #remove non-alphanumeric characters
    txt = re.sub('[\s]+', ' ', txt) #Remove additional white spaces
    return txt

cleantext = []

for item in df['Text']:
    txt = cleanTweets(item)
    cleantext += [txt]

df['clean_text'] = cleantext
df.head(10)

df = df.drop(['Text'],axis=1)
df.head()

In [ ]:
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')
stop[0:10]

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(text)

df['clean_text'] = df['clean_text'].apply(remove_stopwords)
df.head()